### Notebook for initializing polymer systems using a DPD potential

### This notebook is using a new version of mbuild. To install mbuild2.0 in your active environment:
    - git clone git@github.com:mosdef-hub/mbuild.git
    - cd mbuild
    - gh pr checkout 1261
    - pip install -e .

In [1]:
import numpy as np  
import gsd, gsd.hoomd 
import hoomd 
import time
import freud
import matplotlib
import matplotlib_inline
import matplotlib.pyplot as plt
import mbuild as mb
import logging
from mbuild import mBuildLogger
import unyt as u
import random
from mbuild.path import HardSphereRandomWalk, Path
from mbuild.utils.volumes import CuboidConstraint, SphereConstraint, CylinderConstraint
from mbuild.utils.density import rank_points_by_density, find_low_density_point
%matplotlib inline
matplotlib.style.use("ggplot")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

/Users/stephaniemccallum/miniforge3/envs/mupt-examples/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


ImportError: cannot import name 'mBuildLogger' from 'mbuild' (/Users/stephaniemccallum/miniforge3/envs/mupt-examples/lib/python3.12/site-packages/mbuild/__init__.py)

In [14]:
def random_walk(num_mon,num_pol,radius,bond_length,density):
    N = num_mon * num_pol
    L = np.cbrt(N / density)
    cube = CuboidConstraint(center=(0,0,0), Lx=L, Ly=L, Lz=L)
    last_path = None
    print("Starting walk")
    for walk_num in range(num_pol):
        walk_passed = False
        while not walk_passed:
            try:
                if walk_num == 0:
                    initial_point = (0,0,0)
                else:
                  initial_point = find_low_density_point(
                        points=last_path.coordinates,
                        box_min=cube.mins,
                        box_max=cube.maxs,
                        edge_buffer=radius,
                        n_candidates=500,
                    )[np.random.randint(10)]
                
                path = HardSphereRandomWalk(
                    N=num_mon,
                    bead_name=f"_A{walk_num}",
                    radius=radius,
                    volume_constraint=cube,
                    bond_length=bond_length,
                    min_angle=np.pi/2,
                    max_angle=np.pi,
                    max_attempts=1e4,
                    start_from_path=last_path,
                    attach_paths=False,
                    seed=None,
                    initial_point=initial_point,
                    trial_batch_size=100
                )

                last_path = path
                walk_passed = True
            except Exception as e:
                print(e)
    
    polymer_system = last_path.to_compound()
    positions = polymer_system.xyz
    print("Finished walk")

    return positions

def initialize_snapshot_rand_walk(num_pol, num_mon, density=0.85, bond_length=1.0, buffer=0.1):
    '''
    Create a HOOMD snapshot of a cubic box with the number density given by input parameters.
    '''    
    N = num_pol * num_mon
    L = np.cbrt(N / density)  # Calculate box size based on density
    positions = random_walk(num_mon=num_mon,num_pol=num_pol,bond_length=bond_length,radius=0.01,density=density)
    positions = pbc(positions,[L,L,L])
    bonds = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 1):
            bonds.append([start + j, start + j + 1])
    bonds = np.array(bonds)
    frame = gsd.hoomd.Frame()
    frame.particles.types = ['A']
    frame.particles.N = N
    frame.particles.position = positions
    frame.bonds.N = len(bonds)
    frame.bonds.group = bonds
    frame.bonds.types = ['b']
    frame.configuration.box = [L, L, L, 0, 0, 0]
    return frame

def pbc(d,box):
    for i in range(3):
        a = d[:,i]
        a[a < -box[i]/2] += box[i]
        a[a >  box[i]/2] -= box[i]
    return d

def check_bond_length_equilibration(snap,num_mon,num_pol,max_bond_length=1.1,min_bond_length=0.9): #todo
    frame_ds = []
    for j in range(num_pol):
        idx = j*num_mon
        d1 = snap.particles.position[idx:idx+num_mon-1] - snap.particles.position[idx+1:idx+num_mon]
        bond_length = np.linalg.norm(pbc(d1,snap.configuration.box),axis=1)
        frame_ds.append(bond_length)
    max_frame_bond_length = np.max(np.array(frame_ds))
    min_frame_bond_length = np.min(np.array(frame_ds))
    print("max: ",max_frame_bond_length," min: ",min_frame_bond_length)
    if max_frame_bond_length <= max_bond_length and min_frame_bond_length >= min_bond_length:
        print("DPD Simulation Finished.")
        return True
    if max_frame_bond_length > max_bond_length or min_frame_bond_length < min_bond_length:
        return False

def check_inter_particle_distance(snap,minimum_distance=0.95):
    positions = snap.particles.position
    box = snap.configuration.box
    aq = freud.locality.AABBQuery(box,positions)
    aq_query = aq.query(
        query_points=positions,
        query_args=dict(r_min=0.0, r_max=minimum_distance, exclude_ii=True),
    )
    nlist = aq_query.toNeighborList()
    if len(nlist)==0:
        print("Inter-particle separation reached.")
        return True
    else:
        return False

def run_one(A=1000,gamma=1000,k=1000,num_pol=100,num_mon=10,kT=1.0,r_cut = 1.15,bond_length=1.0,dt=0.001,density=0.8,particle_spacing = 1.1
):
    print(num_pol*num_mon)
    print(f"\nRunning with A={A}, gamma={gamma}, k={k}, "
          f"num_pol={num_pol}, num_mon={num_mon}")
    start_time = time.perf_counter()
    frame = initialize_snapshot_rand_walk(num_pol, num_mon, density=density)
    build_stop = time.perf_counter()
    print("Total build time: ", build_stop-start_time)
    harmonic = hoomd.md.bond.Harmonic()
    harmonic.params["b"] = dict(r0=bond_length, k=k)
    integrator = hoomd.md.Integrator(dt=dt)
    integrator.forces.append(harmonic)
    simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=np.random.randint(65535))# TODO seed
    simulation.operations.integrator = integrator 
    simulation.create_state_from_snapshot(frame)
    const_vol = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
    integrator.methods.append(const_vol)
    nlist = hoomd.md.nlist.Cell(buffer=0.4)
    simulation.operations.nlist = nlist
    DPD = hoomd.md.pair.DPD(nlist, default_r_cut=r_cut, kT=kT)
    DPD.params[('A', 'A')] = dict(A=A, gamma=gamma)
    integrator.forces.append(DPD)
    
    simulation.run(0)
    simulation.run(1000)
    snap=simulation.state.get_snapshot()
    
    while not check_bond_length_equilibration(snap,num_mon, num_pol,max_bond_length=particle_spacing): #TODO - work with snapshot instead of gsd?
        check_time = time.perf_counter()
        if (check_time-start_time) > 60:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()

    while not check_inter_particle_distance(snap,minimum_distance=0.95):
        check_time = time.perf_counter()
        if (check_time-start_time) > 60:
            return num_pol*num_mon, 0
        simulation.run(1000)
        snap=simulation.state.get_snapshot()
    
    end_time = time.perf_counter()
    return num_pol*num_mon, end_time - start_time
    
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=50) #A=1000 seems fast, cranking up gamma to 800 speeds up, cranking up k more does too,
#no difference between k=10k and 20k
#OK, with these parameters, we're finishing in under 100 steps
print(f"Finished in time = {s:.2f}s")

5000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=50
Starting walk
('The maximum number attempts allowed have passed, and only ', '13 sucsessful attempts were completed.', 'Try changing the parameters or seed and running again.')
Finished walk
Total build time:  17.317404665984213
max:  1.0299712297609216  min:  0.9729403052464025
DPD Simulation Finished.
Inter-particle separation reached.
Finished in time = 18.34s


In [ ]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=100)
print(f"Finished in time = {s:.2f}s")

In [52]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=1000)
print(f"Finished in time = {s:.2f}s")

100000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=1000
50.0
Total build time:  0.5120189579902217
max:  1.0339326  min:  0.967937
DPD Simulation Finished.
Finished in time = 22.36s


In [6]:
N,s = run_one(A=10000,gamma=800,k=20000,num_pol=200,num_mon=100)
print(f"Finished in time = {s:.2f}s")

20000

Running with A=10000, gamma=800, k=20000, num_pol=200, num_mon=100
Total build time:  0.10955829196609557
max:  1.1032426576806043  min:  0.8423931396572772
max:  1.0819145568037702  min:  0.9104981740053496
DPD Simulation Finished.
Finished in time = 5.31s
